In [75]:
from nltk.tokenize import sent_tokenize,word_tokenize
from collections import Counter
import random
import numpy as np
from sklearn.model_selection import train_test_split


Q2) Parse the dataset into sentences using sentence tokenizer and divide it into 80/20 ratio. Keep 80% dataset for training N-grams and keep 20% for test. You can filter out unnecessary symbols, newlines, etc. You can add symbols <s> and  </s> to mark sentence start and end. 

In [117]:
f=open("aiw.txt","r")
text=f.read().replace("\n"," ")
text=text.lower()

#parse dataset
sent_tokenize_list = sent_tokenize(text)

#divide dataset into 80/20 ratio.
train_dataset,test_dataset=train_test_split(sent_tokenize_list,test_size=0.2,random_state=40)

In [103]:
#parsing the dataset into words
def tokenize(sent_list):
    tokenize_list=[]
    remove=[',','.',"''",'``','--','!','?',';',"'s","(",")","’",'‘','*','[',']',":","“"]
    for i in sent_list:
        tokenize_list.append('<s>')
        words=word_tokenize(i)
        for j in words:
            if j not in remove:
                tokenize_list.append(j)
        tokenize_list.append('</s>')    
    return(tokenize_list)

tokenize_list=tokenize(train_dataset)

# count of unigrams in corpus
unigrams=Counter(tokenize_list)
tokens=0
for i in unigrams.keys():
    tokens+=unigrams[i]

#thus, sze of the vocabulary for given corpus is,
V=len(unigrams)

#bigrams
bigrams=Counter()
for i in range(len(tokenize_list)-1):
    b=(tokenize_list[i],tokenize_list[i+1])
    bigrams[b]=bigrams.get(b,0)+1
#print(bigrams)    
        
#trigrams
trigrams=Counter()
for i in range(len(tokenize_list)-2):
    b=(tokenize_list[i],tokenize_list[i+1],tokenize_list[i+2])
    trigrams[b]=trigrams.get(b,0)+1
#print(trigrams)    

#quadgrams
quadgrams=Counter()
for i in range(len(tokenize_list)-3):
    b=(tokenize_list[i],tokenize_list[i+1],tokenize_list[i+2],tokenize_list[i+3])
    quadgrams[b]=quadgrams.get(b,0)+1
#print(quadgrams)

Q2) Compute MLE for unigram, bigram, trigrams and quadgrams. How many n-grams are possible and how many actually exists? You can either use entire corpus or training corpus.

In [79]:
#maximum likelihood estimation

#unigrams
unig_mle={}
for i in unigrams.keys():
    unig_mle[i]=unigrams[i]/tokens
print(unig_mle)
#bigrams
big_mle={}
for (i,j) in bigrams.keys():
    big_mle[(i,j)]=bigrams[(i,j)]/unigrams[i]
#print(big_mle) 


#trigrams
trig_mle={}
for (i,j,k) in trigrams.keys():
    trig_mle[(i,j,k)]=trigrams[(i,j,k)]/bigrams[(i,j)]


#quadgrams
quad_mle={}
for (i,j,k,l) in quadgrams.keys():
    quad_mle[(i,j,k,l)]=quadgrams[(i,j,k,l)]/trigrams[(i,j,k)]
#print(quad_mle)

{'<s>': 0.033942558746736295, 'wake': 8.703220191470844e-05, 'up': 0.0032201914708442124, 'alice': 0.014273281114012184, 'dear': 0.0011314186248912097, 'said': 0.015796344647519583, 'her': 0.008181026979982594, 'sister': 0.0002610966057441253, 'why': 0.0013489991296779808, 'what': 0.004786771105308964, 'a': 0.021714534377719755, 'long': 0.0010879025239338555, 'sleep': 0.00017406440382941688, 'you': 0.014360313315926894, 've': 0.0016100957354221062, 'had': 0.006353350739773716, 'oh': 0.0016100957354221062, 'i': 0.018320278503046128, 'such': 0.0014795474325500434, 'curious': 0.0005221932114882506, 'dream': 0.00017406440382941688, 'and': 0.0293733681462141, 'she': 0.018798955613577025, 'told': 0.0002175805047867711, 'as': 0.00865970409051349, 'well': 0.0019147084421235858, 'could': 0.002523933855526545, 'remember': 0.0005221932114882506, 'them': 0.003133159268929504, 'all': 0.006353350739773716, 'these': 0.0002175805047867711, 'strange': 0.00013054830287206266, 'adventures': 0.00021758050

# number of possible ngrams

In [80]:
print("number of possible unigrams = ",V)
print("actual number of unigrams =",V)

print("number of possible bigrams = ",V**2)
print("actual number of bigrams =",len(bigrams))

print("number of possible trigrams = ",V**3)
print("actual number of trigrams =",len(trigrams))

print("number of possible quadgrams = ",V**4)
print("actual number of quadgrams =",len(quadgrams))


number of possible unigrams =  2446
actual number of unigrams = 2446
number of possible bigrams =  5982916
actual number of bigrams = 12068
number of possible trigrams =  14634212536
actual number of trigrams = 18893
number of possible quadgrams =  35795283863056
actual number of quadgrams = 21599


# generator

In [81]:
def findnextword(prob,next_word): #function to predict next word using multinomial distribution
    s=0
    for i in prob:
        s+=i
    for i in prob:
        i=i/s
    #print(next_word)
    a=np.random.multinomial(1,prob,size=1).tolist()
    #print(a)
    return next_word[a[0].index(1)]

def bigprediction(cw): #predicted word using bigrams
    prob=[]
    next_word=[]
    for (b1,b2) in big_mle.keys():
        if (b1==cw):
            prob.append(big_mle[(b1,b2)])
            next_word.append(b2)
    return prob,next_word

def trigprediction(cw,cw1): #predicted word using trigrams
    prob=[]
    next_word=[]
    for (t1,t2,t3) in trig_mle.keys():
        if ((t1,t2)==(cw,cw1)):
            prob.append(trig_mle[(t1,t2,t3)])
            next_word.append(t3)
    return prob,next_word

def quadprediction(cw,cw1,cw2): ##predicted word using quadgrams
    prob=[]
    next_word=[]
    for (q1,q2,q3,q4) in quad_mle.keys():
        if ((q1,q2,q3)==(cw,cw1,cw2)):
            prob.append(quad_mle[(q1,q2,q3,q4)])
            next_word.append(q4)
    return prob,next_word

# Sentence generator

In [82]:
def sent_gen(n): #n=2 for bigram, 3 for trigram..
    sent_length=random.randint(8,20) #length of sentence
    sentence=["<s>"]
    length=0
    cw,cw1,cw2="<s>","",""  #current words 0,1,2 for predicting next word
    
    if(n==2):
        while(length<sent_length):
            prob,next_word=bigprediction(cw)
            f=findnextword(prob,next_word)
            
            sentence.append(f)
            cw=f
            length+=1
            if(cw=="</s>"):
                break
    
    if(n==3):
        prob,next_word=bigprediction(cw)  #predict first word using bigram
        f=findnextword(prob,next_word)
        sentence.append(f)
        cw1=f
        length+=1
        
        while(length<sent_length): #predict all other words using trigram
            prob,next_word=trigprediction(cw,cw1)
            f=findnextword(prob,next_word)
            sentence.append(f)
            cw=cw1
            cw1=f
            length+=1
            if(cw1=="</s>"):
                break
    if(n==4):
        prob,next_word=bigprediction(cw) #predict first word using bigram
        f=findnextword(prob,next_word)
        sentence.append(f)
        cw1=f
        
        prob,next_word=trigprediction(cw,cw1) #predict second word using trigram
        f=findnextword(prob,next_word)
        sentence.append(f)
        cw2=f
        length+=2
        
        while(length<sent_length): #predict all other words using quadgram
            prob,next_word=quadprediction(cw,cw1,cw2)
            f=findnextword(prob,next_word)
            sentence.append(f)
            cw=cw1
            cw1=cw2
            cw2=f
            length+=1
            if(cw2=="</s>"):
                break
    s=" ".join(sentence)
    print(s)
print("Bigram generated sentences:")        
for i in range(5):
    sent_gen(2) 
print("\n Trigram generated sentences:")  
for i in range(5):
    sent_gen(3) 
print("\n Quadgram generated sentences:")  
for i in range(5):
    sent_gen(4)  

Bigram generated sentences:
<s> they were obliged to put them word sounded best butter wouldn
<s> the poor child the antipathies i can draw the pigeon raising its mouth but it away consider your
<s> behead that cheshire cat she was going back by the
<s> don t remember what she had been the
<s> anything that it to say that walk with

 Trigram generated sentences:
<s> she pitied him deeply </s>
<s> but perhaps it was even before she gave one sharp kick and waited till she had accidentally upset the
<s> where are you content now said the dormouse </s>
<s> you know this sort in her brother s latin
<s> however she waited for some minutes </s>

 Quadgram generated sentences:
<s> alice thought she had never before seen a rabbit with either a waistcoat-pocket or a watch to take out of
<s> visit either you like they re both mad. but i don
<s> never heard of uglification ” alice ventured to remark </s>
<s> she stretched herself up on tiptoe and peeped over the edge
<s> i quite forgot you didn t 

# log space

In [83]:
def probability(n,sentence):
    words=sentence.split()
    prob=0
    for i in range(len(words)-n):
        try:    #for ngrams that do not exist in ngram list, assume log(p) to be zero.
            if(n==1):
                prob+=np.log(unig_mle[words[i]])
            if(n==2):
                prob+=np.log(big_mle[(words[i],words[i+1])])
            if(n==3):
                prob+=np.log(trig_mle[(words[i],words[i+1],words[i+2])])
            if(n==4):
                prob+=np.log(trig_mle[(words[i],words[i+1],words[i+2],words[i+3])])
        except:
            continue
    print(10**prob)
sentence=sent_tokenize_list[k_train+20]
probability(3,sentence)

0.0011969606806737502


# add 1 smoothing

In [84]:

#for mle's, add 1 to count of bigram and V to total count.

big_mle1={}
bigrams1={} #effective count after smoothing
for (i,j) in bigrams.keys():
    big_mle1[(i,j)]=(bigrams[(i,j)]+1)/(unigrams[i]+V)
    bigrams1[(i,j)]=unigrams[i]*big_mle1[(i,j)]
#print(big_mle1)
l=[(bigrams[(i,j)]-bigrams1[(i,j)],i,j) for (i,j) in bigrams.keys()]
l.sort(reverse=True)
print("   Bigram","              Difference in count for mle and add one")
for i in range(9):
    print(l[i][1:],":        ",l[i][0])


   Bigram               Difference in count for mle and add one
('</s>', '<s>') :         590.4073155610663
('said', 'the') :         140.06514773940904
('of', 'the') :         85.21783574198096
('said', 'alice') :         82.5941616233535
('<s>', 'i') :         68.75573465592065
('in', 'a') :         60.91269258987528
('it', 'was') :         52.30266343825666
('in', 'the') :         51.04255319148936
('and', 'the') :         49.94200576738225


The drastic change in counts post smoothing is due to unequal distribution of the added one count to various bigrams. For bigrams with large counts, adding just 1 and dividing by V affects the effective count the most, leading to maximum difference.

# good Turing 

In [96]:
def good_turing(n):
    #adjusted counts
    #N[c]= number of bigrams seen exactly c times 
    N={}
    N[0]=0
    for (i,j) in bigrams:
        c=bigrams[(i,j)]
        N[c]=N.get(c,0)+1
        N[0]+=c
    print(N[0])
    
    #Turing_count[c]=effective turing count of bigrams seen exactly c times
    Turing_count={}
    Turing_count[11]=N[11] #for the last element in N.
    c=0
    avg=0
    while (c<10):
        Turing_count[c]= ((c+1)*N[c+1])/N[c]
        avg+=(c-Turing_count[c])
        print("c:", c, ", d:", c-Turing_count[c])
        c+=1
    d=avg/10
    print("Value of d for good turing smoothing is", d) 
good_turing(0)

22979
c: 0 , d: -0.39875538535184296
c: 1 , d: 0.6795809232784022
c: 2 , d: 0.9414168937329701
c: 3 , d: 0.9382239382239383
c: 4 , d: 1.3408239700374533
c: 5 , d: 0.436619718309859
c: 6 , d: 1.1388888888888893
c: 7 , d: -0.040000000000000036
c: 8 , d: 2.954545454545454
c: 9 , d: -0.18918918918918948
Value of d for good turing smoothing is 0.7802155212475934


# perplexity

In [140]:
#test_dataset=sent_tokenize_list[k_train::]
test_words=tokenize(test_dataset)
#print(len(test_dataset))
#perplexity for add 1 smoothing
def perplexity(test_words,big_mle1):
    p=1
    for i in range(len(test_words)-2):
        try:
            p*=(1/(big_mle1[(test_words[i],test_words[i+1])]))**(1/len(test_words))
            #print(p,"1\n")
        except:
            prob=1/(unigrams[i]+V)
            p*=(1/prob)**(1/len(test_words))
            #print(p,"2\n")
    
    return p
print("perplexity for add 1 smoothing",perplexity(test_words,big_mle1))

N=[]
N.append(22979)
#perplexity for good turing
def perplexityturing(test_words,d,bigrams):
    p=1
    #print(d)
    for i in range(len(test_words)-2):
        try:
            b=(test_words[i],test_words[i+1])
            c=bigrams[b]
            if(c>d):
                cstar=c-d
            else:
                cstar=d-c
            prob=cstar/N[0]
            #print(cstar)
        except:
            prob=1.0/N[0]
            #print("hi")
        p*=(1/prob)**(1/len(test_words))
    
    return p
print("perplexity for good turing smoothing",perplexityturing(test_words,0.780,bigrams))



perplexity for add 1 smoothing 522.5550762319765
perplexity for good turing smoothing 12194.885974612633
